# Upload explainer to MinIO artifact store

In [ ]:
from kfp.components import create_component_from_func, InputPath
from typing import NamedTuple

%load_ext lab_black

BASE_IMAGE = "quay.io/ibm/kubeflow-notebook-image-ppc64le:latest"


def upload_explainer(
    file_dir: InputPath(str),
    project_name: str,
    model_version: int,
    minio_url: str = "http://minio-service.kubeflow:9000",
    minio_secret: str = "mlpipeline-minio-artifact",
    export_bucket: str = "projects",
) -> NamedTuple("UploadOutput", [("s3_address", str), ("triton_s3_address", str)],):
    """Uploads an explainer file to MinIO artifact store."""

    import boto3
    import botocore
    from botocore.client import Config
    from collections import namedtuple
    from kubernetes import client, config
    import logging
    import sys

    logging.basicConfig(
        stream=sys.stdout,
        level=logging.INFO,
        format="%(levelname)s %(asctime)s: %(message)s",
    )
    logger = logging.getLogger()
    
    try:
        model_version = int(model_version)
    except ValueError as e:
        logger.warning(
            "Could not parse model version. Continuing with default value 1..."
        )
        model_version = 1

    def get_minio_client(minio_secret):
        import base64
        from kubernetes.client.rest import ApiException

        def get_current_namespace():
            SA_NAMESPACE = "/var/run/secrets/kubernetes.io/serviceaccount/namespace"
            with open(SA_NAMESPACE) as f:
                return f.read()

        def decode(text):
            return base64.b64decode(text).decode("utf-8")

        config.load_incluster_config()
        api_client = client.ApiClient()

        try:
            secret = client.CoreV1Api(api_client).read_namespaced_secret(
                minio_secret, get_current_namespace()
            )

            minio_user = decode(secret.data["accesskey"])
            minio_pass = decode(secret.data["secretkey"])

            return boto3.session.Session().resource(
                service_name="s3",
                endpoint_url=minio_url,
                aws_access_key_id=minio_user,
                aws_secret_access_key=minio_pass,
                config=Config(signature_version="s3v4"),
            )
        except ApiException as e:
            if e.status == 404:
                logger.error(
                    "Failed to get secret 'mlpipeline-minio-artifact', which is needed for communicating with MinIO!"
                )
            raise Exception(e)

    # see: https://stackoverflow.com/a/47565719/2625096
    def bucket_exists(minio_client, bucket):
        try:
            minio_client.meta.client.head_bucket(Bucket=bucket.name)
            return True
        except botocore.exceptions.ClientError as e:
            error_code = int(e.response["Error"]["Code"])
            if error_code == 403:
                # Forbidden Access -> Private Bucket
                return True
            elif error_code == 404:
                return False

    # see: https://stackoverflow.com/questions/57957585/how-to-check-if-a-particular-directory-exists-in-s3-bucket-using-python-and-boto
    def folder_exists(minio_client, bucket: str, path: str) -> bool:
        path = path.rstrip("/")
        resp = minio_client.meta.client.list_objects_v2(
            Bucket=bucket.name, Prefix=path, Delimiter="/", MaxKeys=1
        )
        return "CommonPrefixes" in resp

    def get_versions(minio_client, bucket: str, path: str):
        from pathlib import Path

        path = path.rstrip("/")
        response = minio_client.meta.client.list_objects_v2(
            Bucket=bucket.name, Prefix=path
        )
        versions = set()
        for content in response.get("Contents", []):
            versions.add(int(Path(content["Key"]).parts[2]))
        return versions

    logger.info(f"Establishing MinIO connection to '{minio_url}'...")
    minio_client = get_minio_client(minio_secret)

    # Create export bucket if it does not yet exist
    bucket = minio_client.Bucket(export_bucket)
    if not bucket_exists(minio_client, bucket):
        logger.info(f"Creating bucket '{export_bucket}'...")
        minio_client.create_bucket(Bucket=bucket.name)

    explainer_path = f"explainer/{model_version}"
    s3_address = f"{minio_url}/{export_bucket}/{project_name}"
    triton_s3_address = f"{s3_address}/{explainer_path}"

    logger.info("Saving onnx file to MinIO...")
    logger.info(f"s3 address: {s3_address}")
    logger.info(f"triton address: {triton_s3_address}")
    bucket.upload_file(
        f"{file_dir}/explainer.dill", f"{project_name}/{explainer_path}/explainer.dill"
    )
    bucket.upload_file(
        f"{file_dir}/meta.dill", f"{project_name}/{explainer_path}/meta.dill"
    )

    logger.info("Finished.")
    out_tuple = namedtuple("UploadOutput", ["s3_address", "triton_s3_address"])
    return out_tuple(s3_address, triton_s3_address)


upload_explainer_comp = create_component_from_func(
    func=upload_explainer, output_component_file="component.yaml", base_image=BASE_IMAGE
)